In [76]:
# !pip install deep_translator
# !pip install langdetect
# !pip install langid

In [77]:
import pandas as pd
import langid
import requests
import re
import pickle


In [78]:

## importing of the data
users_data = pd.read_csv("data/external/users.csv").convert_dtypes()
ratings_data = pd.read_csv("data/external/ratings.csv").convert_dtypes()
books_data = pd.read_csv("data/external/books.csv", low_memory=False)

In [79]:
# put the title and author all to upper case
books_data['Book-Title'] = books_data['Book-Title'].str.upper()
books_data['Book-Author'] = books_data['Book-Author'].str.upper()
books_data['Book-Title'] = books_data['Book-Title'].astype(str)
books_data['Publisher'] = books_data['Publisher'].str.upper()
books_data = books_data[['ISBN', 'Book-Author', 'Book-Title','Year-Of-Publication','Publisher']]

pattern = r'[():]'
books_data['Book-Title'] = books_data['Book-Title'].apply(lambda x:re.sub(pattern, '', x))
books_data['Language'] = None

In [80]:
# Checking  for  null value Book-Author
books_data[books_data['Book-Author'].isna()]


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
118033,0751352497,NaN,A+ QUIZ MASTERS01 EARTH,1999,DORLING KINDERSLEY,None
187689,9627982032,NaN,THE CREDIT SUISSE GUIDE TO MANAGING YOUR PERSO...,1995,EDINBURGH FINANCIAL PUBLISHING,None


In [81]:
# updating the null value
books_data.loc[187689, 'Book-Author'] = str.upper('Larissa Anne Downes')

In [82]:
# Checking  for  null value Book-Title
books_data[books_data['Book-Title'].isna()]


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language


In [83]:
# Checking  for  null value Year-Of-Publication
books_data[books_data['Year-Of-Publication'].isna()]


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language


In [84]:
# Checking  for  null value Publisher
books_data[books_data['Publisher'].isna()]

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
128890,193169656X,ELAINE CORVIDAE,TYRANT MOON,2002,NaN,None
129037,1931696993,LINNEA SINCLAIR,FINDERS KEEPERS,2001,NaN,None


In [85]:
books_data.loc[128890, 'Publisher'] = str.upper('NovelBooks, Inc.')
books_data.loc[129037, 'Publisher'] = str.upper('CreateSpace Independent Publishing Platform')

In [87]:
books_data['Year-Of-Publication'].unique()

array(['2002', '2001', '1991', '1999', '2000', '1993', '1996', '1988',
       '2004', '1998', '1994', '2003', '1997', '1983', '1979', '1995',
       '1982', '1985', '1992', '1986', '1978', '1980', '1952', '1987',
       '1990', '1981', '1989', '1984', '0', '1968', '1961', '1958',
       '1974', '1976', '1971', '1977', '1975', '1965', '1941', '1970',
       '1962', '1973', '1972', '1960', '1966', '1920', '1956', '1959',
       '1953', '1951', '1942', '1963', '1964', '1969', '1954', '1950',
       '1967', '2005', '1957', '1940', '1937', '1955', '1946', '1936',
       '1930', '2011', '1925', '1948', '1943', '1947', '1945', '1923',
       '2020', '1939', '1926', '1938', '2030', '1911', '1904', '1949',
       '1932', '1928', '1929', '1927', '1931', '1914', '2050', '1934',
       '1910', '1933', '1902', '1924', '1921', '1900', '2038', '2026',
       '1944', '1917', '1901', '2010', '1908', '1906', '1935', '1806',
       '2021', '2012', '2006', 'DK Publishing Inc', 'Gallimard', '1909',
       

In [88]:
books_data.loc[books_data['Year-Of-Publication'].isin(['DK Publishing Inc','Gallimard'])] 

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
209538,078946697X,2000,"DK READERS CREATING THE X-MEN, HOW IT ALL BEGA...",DK Publishing Inc,HTTP://IMAGES.AMAZON.COM/IMAGES/P/078946697X.0...,None
220731,2070426769,2003,"PEUPLE DU CIEL, SUIVI DE 'LES BERGERS\"";JEAN-M...",Gallimard,HTTP://IMAGES.AMAZON.COM/IMAGES/P/2070426769.0...,None
221678,0789466953,2000,"DK READERS CREATING THE X-MEN, HOW COMIC BOOKS...",DK Publishing Inc,HTTP://IMAGES.AMAZON.COM/IMAGES/P/0789466953.0...,None


In [89]:
books_data.loc[209538,'Year-Of-Publication'] = 2000
books_data.loc[220731,'Year-Of-Publication'] = 2003
books_data.loc[221678,'Year-Of-Publication'] = 2000

In [91]:
books_data['Year-Of-Publication'].unique()

array(['2002', '2001', '1991', '1999', '2000', '1993', '1996', '1988',
       '2004', '1998', '1994', '2003', '1997', '1983', '1979', '1995',
       '1982', '1985', '1992', '1986', '1978', '1980', '1952', '1987',
       '1990', '1981', '1989', '1984', '0', '1968', '1961', '1958',
       '1974', '1976', '1971', '1977', '1975', '1965', '1941', '1970',
       '1962', '1973', '1972', '1960', '1966', '1920', '1956', '1959',
       '1953', '1951', '1942', '1963', '1964', '1969', '1954', '1950',
       '1967', '2005', '1957', '1940', '1937', '1955', '1946', '1936',
       '1930', '2011', '1925', '1948', '1943', '1947', '1945', '1923',
       '2020', '1939', '1926', '1938', '2030', '1911', '1904', '1949',
       '1932', '1928', '1929', '1927', '1931', '1914', '2050', '1934',
       '1910', '1933', '1902', '1924', '1921', '1900', '2038', '2026',
       '1944', '1917', '1901', '2010', '1908', '1906', '1935', '1806',
       '2021', '2012', '2006', 2000, 2003, '1909', '2008', '1378', '1919',
     

In [99]:
# find the Book-Author, which is only number
numeric_rows = books_data['Book-Author'].str.contains('^\d+$')
print(books_data[numeric_rows.fillna(False)])


              ISBN Book-Author  \
209538  078946697X        2000   
220731  2070426769        2003   
221678  0789466953        2000   

                                               Book-Title Year-Of-Publication  \
209538  DK READERS CREATING THE X-MEN, HOW IT ALL BEGA...                2000   
220731  PEUPLE DU CIEL, SUIVI DE 'LES BERGERS\";JEAN-M...                2003   
221678  DK READERS CREATING THE X-MEN, HOW COMIC BOOKS...                2000   

                                                Publisher Language  
209538  HTTP://IMAGES.AMAZON.COM/IMAGES/P/078946697X.0...     None  
220731  HTTP://IMAGES.AMAZON.COM/IMAGES/P/2070426769.0...     None  
221678  HTTP://IMAGES.AMAZON.COM/IMAGES/P/0789466953.0...     None  


In [100]:
books_data.loc[209538, 'Book-Author'] = str.upper('Teitelbaum, Michael')
books_data.loc[220731, 'Book-Author'] = str.upper('Le Clezio, Jean-Marie Gustave')
books_data.loc[221678, 'Book-Author'] = str.upper('Buckley Jr., James')

In [ ]:
books_data['Language'] = books_data['Book-Title'].apply(lambda x: langid.classify(x)[0])
books_data['Language'] = books_data['Language'].apply(lambda x: str.upper(x))

In [121]:
books_data[books_data.duplicated()]

Empty DataFrame
Columns: [ISBN, Book-Author, Book-Title, Year-Of-Publication, Publisher, Language]
Index: []


In [112]:
ratings_data[ratings_data['User-ID'].isna()]

,User-ID,ISBN,Book-Rating


In [114]:
ratings_data[ratings_data['ISBN'].isna()]

,User-ID,ISBN,Book-Rating


In [115]:
ratings_data[ratings_data['Book-Rating'].isna()]

,User-ID,ISBN,Book-Rating


In [119]:
ratings_data[ratings_data.duplicated()]

,User-ID,ISBN,Book-Rating


In [122]:
users_data[users_data['User-ID'].isna()]

,User-ID,Location,Age


In [123]:
users_data[users_data['Location'].isna()]

,User-ID,Location,Age


In [124]:
users_data[users_data['Age'].isna()]

,User-ID,Location,Age
0,1,"nyc, new york, usa",<NA>
2,3,"moscow, yukon territory, russia",<NA>
4,5,"farnborough, hants, united kingdom",<NA>
6,7,"washington, dc, usa",<NA>
7,8,"timmins, ontario, canada",<NA>
...,...,...,...
278849,278850,"sergnano, lombardia, italy",<NA>
278853,278854,"portland, oregon, usa",<NA>
278855,278856,"brampton, ontario, canada",<NA>
278856,278857,"knoxville, tennessee, usa",<NA>


In [125]:
with open('data/interim/books_data.pkl', 'wb') as f:
    pickle.dump(books_data, f)

with open('data/interim/ratings_data.pkl', 'wb') as f:
    pickle.dump(ratings_data, f)

with open('data/interim/users_data.pkl', 'wb') as f:
    pickle.dump(users_data, f)